# cmdrnafold - Interactive Examples

This notebook demonstrates the capabilities of the `cmdrnafold` package for RNA secondary structure prediction using ViennaRNA commandline tools.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/retospect/cmdrnafold/blob/main/examples/cmdrnafold_examples.ipynb)

## Installation

First, let's install the required packages:

In [ ]:
# Install cmdrnafold
!pip install cmdrnafold

# Install ViennaRNA (Ubuntu/Debian)
# !apt-get update && apt-get install -y vienna-rna

# For other systems, please install ViennaRNA manually:
# macOS: brew install viennarna
# Windows: Download from https://www.tbi.univie.ac.at/RNA/

## Basic Usage - Synchronous Interface

The synchronous interface provides a drop-in replacement for the ViennaRNA Python bindings:

In [ ]:
from cmdrnafold import RNA

# Example RNA sequence
sequence = "CGCAGGGAUACCCGCG"
print(f"Sequence: {sequence}")

# Create fold compound
fc = RNA.fold_compound(sequence)

# Compute minimum free energy structure
structure, mfe = fc.mfe()

print(f"Structure: {structure}")
print(f"MFE: {mfe:.2f} kcal/mol")
print(f"Formatted: {structure} [{mfe:6.2f}]")

## Advanced Usage - Asynchronous Interface

The asynchronous interface allows for concurrent processing of multiple sequences:

In [ ]:
import asyncio
from cmdrnafold import fold_compound
import time

# Multiple RNA sequences for testing
sequences = [
    "CGCAGGGAUACCCGCG",
    "AAAUUUGGGCCCUUUU", 
    "GGGCCCAAAUUUGGGCCC",
    "UUUUAAAACCCCGGGG",
    "GCGCGCGCGCGCGCGC"
]

async def fold_sequences_async(sequences):
    """Fold multiple sequences concurrently."""
    print("Folding sequences asynchronously...")
    start_time = time.time()
    
    # Create tasks for concurrent execution
    tasks = []
    for seq in sequences:
        fc = fold_compound(seq)
        tasks.append(fc.mfe())
    
    # Execute all tasks concurrently
    results = await asyncio.gather(*tasks)
    
    end_time = time.time()
    print(f"Completed in {end_time - start_time:.2f} seconds\n")
    
    return results

def fold_sequences_sync(sequences):
    """Fold multiple sequences sequentially."""
    print("Folding sequences synchronously...")
    start_time = time.time()
    
    results = []
    for seq in sequences:
        fc = RNA.fold_compound(seq)
        results.append(fc.mfe())
    
    end_time = time.time()
    print(f"Completed in {end_time - start_time:.2f} seconds\n")
    
    return results

# Compare async vs sync performance
print("Performance comparison:")
print("=" * 50)

# Async version
async_results = await fold_sequences_async(sequences)

# Sync version
sync_results = fold_sequences_sync(sequences)

# Display results
print("Results:")
print("-" * 50)
for i, (seq, (structure, mfe)) in enumerate(zip(sequences, async_results)):
    print(f"Sequence {i+1}: {seq}")
    print(f"Structure:  {structure}")
    print(f"MFE:        {mfe:6.2f} kcal/mol")
    print()

## Error Handling and Input Validation

The library provides comprehensive error handling and input validation:

In [ ]:
from cmdrnafold import RNA, RNAFoldError

# Test various invalid inputs
invalid_sequences = [
    "",  # Empty sequence
    "AUGCX",  # Invalid nucleotide
    "123456",  # Numbers
    "AUGC AUGC",  # Spaces
    "AUGC-AUGC",  # Hyphens
    None,  # None value
]

print("Testing input validation:")
print("=" * 40)

for i, seq in enumerate(invalid_sequences):
    try:
        print(f"Test {i+1}: {repr(seq)}")
        fc = RNA.fold_compound(seq)
        structure, mfe = fc.mfe()
        print(f"  ✗ Unexpected success: {structure} [{mfe:.2f}]")
    except RNAFoldError as e:
        print(f"  ✓ Correctly caught error: {e}")
    except Exception as e:
        print(f"  ✓ Caught other error: {type(e).__name__}: {e}")
    print()

# Test valid sequences with different cases
print("Testing case handling:")
print("-" * 40)

test_sequences = [
    "AUGC",  # Uppercase
    "augc",  # Lowercase
    "AuGc",  # Mixed case
]

for seq in test_sequences:
    try:
        fc = RNA.fold_compound(seq)
        structure, mfe = fc.mfe()
        print(f"Input: {seq:8} → Structure: {structure} [{mfe:6.2f}]")
    except RNAFoldError as e:
        print(f"Input: {seq:8} → Error: {e}")

## Biological Examples

Let's analyze some biologically relevant RNA sequences:

In [ ]:
# Biologically relevant RNA sequences
bio_sequences = {
    "tRNA-like": "GCGGAUUUAGCUCAGUUGGGAGAGCGCCAGACUGAAGAUCUGGAGGUCCUGUGUUCGAUCCACAGAAUUCGCA",
    "Hairpin loop": "CGCAGGGAUACCCGCG",
    "Stem-loop": "GGGGAAAACCCC",
    "Internal loop": "GGGCCCAAAUUUGGGCCC",
    "Bulge loop": "GGGCCCAAUUUGGGCCC",
    "Multi-branch": "GGGAAACCCUUUGGGAAACCC",
}

print("Biological RNA Structure Analysis")
print("=" * 50)

for name, sequence in bio_sequences.items():
    try:
        fc = RNA.fold_compound(sequence)
        structure, mfe = fc.mfe()
        
        # Calculate some basic statistics
        length = len(sequence)
        gc_content = (sequence.count('G') + sequence.count('C')) / length * 100
        paired_bases = structure.count('(') + structure.count(')')
        pairing_ratio = paired_bases / length * 100
        
        print(f"\n{name}:")
        print(f"  Sequence:     {sequence}")
        print(f"  Structure:    {structure}")
        print(f"  Length:       {length} nt")
        print(f"  GC content:   {gc_content:.1f}%")
        print(f"  Paired bases: {paired_bases}/{length} ({pairing_ratio:.1f}%)")
        print(f"  MFE:          {mfe:.2f} kcal/mol")
        print(f"  MFE/nt:       {mfe/length:.3f} kcal/mol/nt")
        
    except RNAFoldError as e:
        print(f"\n{name}: Error - {e}")

## Visualization Helper

Let's create a simple visualization of the RNA structures:

In [ ]:
def visualize_rna_structure(sequence, structure, name="RNA"):
    """
    Create a simple text-based visualization of RNA structure.
    """
    print(f"\n{name} Structure Visualization:")
    print("-" * (len(name) + 25))
    
    # Print sequence with position numbers
    print("Position: ", end="")
    for i in range(0, len(sequence), 10):
        print(f"{i+1:>10}", end="")
    print()
    
    print("Sequence: ", end="")
    for i, base in enumerate(sequence):
        if i % 10 == 0 and i > 0:
            print(" ", end="")
        print(base, end="")
    print()
    
    print("Structure:", end="")
    for i, symbol in enumerate(structure):
        if i % 10 == 0 and i > 0:
            print(" ", end="")
        print(symbol, end="")
    print()
    
    # Analyze structure elements
    stems = structure.count('(')
    loops = structure.count('.')
    
    print(f"\nStructure Analysis:")
    print(f"  Stem regions: {stems} base pairs")
    print(f"  Loop regions: {loops} unpaired bases")
    print(f"  Total length: {len(sequence)} nucleotides")

# Visualize some examples
examples = [
    ("CGCAGGGAUACCCGCG", "Simple Hairpin"),
    ("GGGCCCAAAUUUGGGCCC", "Internal Loop"),
    ("GGGGAAAACCCCUUUUGGGGAAAACCCC", "Two Hairpins"),
]

for sequence, name in examples:
    try:
        fc = RNA.fold_compound(sequence)
        structure, mfe = fc.mfe()
        visualize_rna_structure(sequence, structure, name)
        print(f"MFE: {mfe:.2f} kcal/mol")
    except RNAFoldError as e:
        print(f"Error with {name}: {e}")

## Performance Benchmarking

Let's benchmark the performance with sequences of different lengths:

In [ ]:
import time
import random

def generate_random_rna(length):
    """Generate a random RNA sequence of given length."""
    bases = ['A', 'U', 'G', 'C']
    return ''.join(random.choice(bases) for _ in range(length))

def benchmark_folding(lengths, num_sequences=5):
    """Benchmark RNA folding for different sequence lengths."""
    print("Performance Benchmark")
    print("=" * 50)
    print(f"{'Length':<8} {'Sequences':<10} {'Avg Time':<12} {'Total Time':<12}")
    print("-" * 50)
    
    for length in lengths:
        sequences = [generate_random_rna(length) for _ in range(num_sequences)]
        
        start_time = time.time()
        
        for seq in sequences:
            try:
                fc = RNA.fold_compound(seq)
                structure, mfe = fc.mfe()
            except RNAFoldError:
                pass  # Skip sequences that fail
        
        end_time = time.time()
        total_time = end_time - start_time
        avg_time = total_time / num_sequences
        
        print(f"{length:<8} {num_sequences:<10} {avg_time:<12.4f} {total_time:<12.4f}")

# Benchmark different sequence lengths
test_lengths = [10, 20, 50, 100, 200]
benchmark_folding(test_lengths, num_sequences=3)

## Summary and Best Practices

### Key Features Demonstrated:

1. **Synchronous Interface**: Drop-in replacement for ViennaRNA Python bindings
2. **Asynchronous Interface**: Concurrent processing for better performance
3. **Error Handling**: Comprehensive input validation and error reporting
4. **Type Safety**: Full type annotations for better development experience
5. **Performance**: Efficient processing of multiple sequences

### Best Practices:

1. **Use async interface** for processing multiple sequences
2. **Always handle RNAFoldError** for robust applications
3. **Validate input sequences** before processing large batches
4. **Consider sequence length** for performance planning
5. **Use appropriate interface** (sync vs async) based on your use case

### Next Steps:

- Explore the [GitHub repository](https://github.com/retospect/cmdrnafold) for more examples
- Check the [API documentation](https://github.com/retospect/cmdrnafold#api-reference)
- Report issues or contribute improvements
- Star the repository if you find it useful! ⭐